## Career quiz
This notebook is meant to suggest a user viable career paths // ideas based on some QnA that evolves as the user is answering questions.

In [58]:
import json
from openai import OpenAI
from dotenv import load_dotenv

In [59]:
# Load environment variables from .env file
load_dotenv()
# load openai key
client = OpenAI()

In [60]:
# set up fcn to call openai
def llm(user_prompt,system_prompt="you are a helpful assistant",model="gpt-4o-mini",temperature=0.5):
    ''' this function calls the openAI api and feeds it user/system prompts'''
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature,
        )
    return response.choices[0].message.content

In [61]:
def load_prompts(user_prompt_filename="user_prompt.txt",system_prompt_filename="system_prompt.txt"):
    # read the prompts from the txt files
    with open(user_prompt_filename, "r") as f:
        QNA_USER_PROMPT = f.read()

    with open(system_prompt_filename, "r") as f:
        QNA_SYSTEM_PROMPT = f.read()

    return QNA_USER_PROMPT, QNA_SYSTEM_PROMPT

In [62]:
def load_quiz_files(quiz_filename="career_quiz.json",responses_filename="user_response.json"):
    with open(quiz_filename, "r") as f:
        career_quiz = json.load(f)

    with open(responses_filename, "r") as f:
        user_response = json.load(f)

    return career_quiz, user_response

In [63]:
def format_prompts(QNA_USER_PROMPT,QNA_SYSTEM_PROMPT,career_quiz=[],user_response=[]):
    ''' this function formats both the user and system prompt given quizz/answer selections so far'''

    # format the quiz and the answer
    qna_user_prompt = QNA_USER_PROMPT.format(user_response=json.dumps(user_response, indent=2))
    qna_system_prompt = QNA_SYSTEM_PROMPT.format(career_quiz=json.dumps(career_quiz, indent=2))

    return qna_user_prompt, qna_system_prompt


## MAIN

In [64]:
# set up fake JSON input
data = {
    'career_quiz': [],
    'user_response': []
}

In [65]:
# load the prompts
QNA_USER_PROMPT, QNA_SYSTEM_PROMPT = load_prompts()

# read any input data from the system
if not (data['career_quiz'] and data['user_response']):
    # load the initial career quiz and response
    career_quiz, user_response = load_quiz_files()
else:
    career_quiz = data['career_quiz']
    user_response = data['user_response']


In [66]:
# Debugging purposes only
QNA_SYSTEM_PROMPT = """ 

You are a career counselor guiding a user through a personalized quiz called 'CAREER_QUIZ'.

Important context about the user:

- The user is actively open to pursuing a new online degree.
- All degrees are available online, so **do NOT ask about willingness to study, take classes, or preferred learning formats**.

Your mission is to generate thoughtful, non-repetitive follow-up questions that help match the user to **a diverse set of potential career paths**, including *emerging, interdisciplinary, and less commonly suggested fields*.

Examples of often-overlooked careers to consider surfacing:
- Linguist, translator, anthropologist, archivist, ethicist, policy analyst, forensic psychologist, UX researcher, conservationist, public health educator, etc.

Focus your questions on:
- Passions and interests (e.g. topics, activities, causes)
- Personality traits (e.g. curiosity, empathy, risk-taking)
- Strengths and abilities (e.g. languages, math, problem solving)
- Career goals and life stage (e.g. flexibility, impact, growth)
- Preferred work settings (e.g. remote vs hands-on, solo vs team, structured vs creative)
- Industry preferences (e.g. healthcare, technology, arts, public sector, startups)
- Motivations (e.g. stability, purpose, autonomy, income potential)

Avoid:
- Repeating themes (e.g., asking 3 variations of "do you like working with people?")
- Defaulting to generic careers like teacher, software engineer, or business analyst *unless the user strongly signals that path*

You receive the current quiz state under <CAREER_QUIZ> with questions (IDs 1–10) and user answers.

Your job:

1. If **10 questions already exist (IDs 1–10)**, STOP asking and output ONLY a career recommendation JSON (see below).
2. If there are **fewer than 10 questions**, generate **exactly ONE new follow-up question**, using the smallest unused ID from 1–10.
3. **Never reuse or skip question IDs. Never use an ID > 10.**
4. All output must be a single JSON object with no extra text, no Markdown, and no formatting.

**Output formats:**

- When generating a follow-up question:

{{
  "id": <smallest unused integer from 1 to 10>,
  "question": "A single clear follow-up question?",
  "options": ["Option A", "Option B", "Option C"],
  "multiple_answers": true or false
}}

- When generating recommendations (after 10 questions):

{{
  "career_recommendations": ["Career A", "Career B", "Career C", ..., "Career N"],
  "reasoning": "Brief explanation (1–2 sentences) linking user answers to recommendations."
}}

**MANDATORY RULES:**
- Recommend **5–10 careers**, no fewer, no more.
- NEVER generate any question with an "id" greater than 10.
- If 10 questions exist, **only output recommendations**.
- Violation will result in a $20,000 USD penalty.

<CAREER_QUIZ>
{career_quiz}
</CAREER_QUIZ>

"""

In [125]:
# format prompts and get response
qna_user_prompt, qna_system_prompt = format_prompts(QNA_USER_PROMPT,QNA_SYSTEM_PROMPT,career_quiz,user_response)
response = llm(user_prompt = qna_user_prompt,system_prompt=qna_system_prompt,model="gpt-4o-mini",temperature=0.5)

In [126]:
# extract new question
new_question = json.loads(response)
print(new_question)


{'career_recommendations': ['UX Researcher', 'Digital Archivist', 'Policy Analyst', 'Forensic Psychologist', 'Ethicist', 'Conservationist', 'Public Health Educator', 'Linguist', 'Creative Technologist'], 'reasoning': 'Your interests in technology and arts, combined with your analytical and creative personality traits, suggest careers that bridge these fields. Roles like UX researcher and digital archivist leverage your technical skills and desire for creative expression, while positions like policy analyst and forensic psychologist align with your problem-solving abilities and curiosity.'}


In [121]:

# pretend the user selected and answer
new_response = {
    "id": new_question['id'],
    "question": new_question['question'],
    "selections": [new_question['options'][i] for i in [3]]
}
print(new_response)


{'id': 10, 'question': 'What motivates you most in your career?', 'selections': ['High income potential']}


In [122]:
# append to the career quiz and the user_response
career_quiz.append(new_question)
user_response.append(new_response)


In [123]:
# output the updated career quiz and user_response as json bits
data = {
    'career_quiz': career_quiz,
    'user_response': user_response
}

In [124]:
data

{'career_quiz': [{'id': 1,
   'question': 'Where are you in your professional development?',
   'options': ['high-school student',
    'college student',
    'graduate student',
    'early-career professional',
    'late career professional',
    'lifelong learner'],
   'multiple_answers': False},
  {'id': 2,
   'question': 'What topics or causes are you most passionate about?',
   'options': ['Environmental issues',
    'Social justice',
    'Technology and innovation',
    'Health and wellness',
    'Arts and culture'],
   'multiple_answers': True},
  {'id': 3,
   'question': 'Which personality traits best describe you?',
   'options': ['Curious',
    'Empathetic',
    'Analytical',
    'Creative',
    'Detail-oriented'],
   'multiple_answers': True},
  {'id': 4,
   'question': 'What strengths or abilities do you possess that you feel could be valuable in a career?',
   'options': ['Strong writing skills',
    'Technical skills (e.g., coding, design)',
    'Problem-solving abilities'